In [1]:
import gmsh 
import numpy as np

gmsh.initialize()

# cubo 

Lx = 10
Ly = 10
h  = 1 
box = gmsh.model.occ.addBox(0, 0, -h/2, Lx, Ly, h)

# synco 

# cilindros 

r = 0.8 

def createcyl(i):
    c1 = gmsh.model.occ.addCylinder(0, 0, 0, 0, 0, Lx, r/2)
    # rotate
    gmsh.model.occ.rotate([(3, c1)], 0, 0, 0, 0, r, 0, np.pi/2)
    # tranlate y 
    gmsh.model.occ.translate([(3, c1)], 0, i,0)
    gmsh.model.occ.synchronize()
    return c1

cs = []
for i in range(1, int(Ly/h)):
    ic = createcyl(i*h)
    cs.append(ic)

# cut box and cylinders 
gmsh.model.occ.synchronize()

gmsh.model.occ.cut([(3, box)], [(3, c) for c in cs],removeTool=False)

gmsh.model.occ.synchronize()

# mesh 
# size
gmsh.option.setNumber("Mesh.CharacteristicLengthMin", 0.1)
gmsh.option.setNumber("Mesh.CharacteristicLengthMax", 0.5)
gmsh.option.setNumber("Mesh.MeshSizeFromCurvature", 15)
#
gmsh.model.mesh.generate(3)
# optimize
gmsh.model.mesh.optimize("Netgen")
# set order 2 
gmsh.model.mesh.setOrder(2)

# input file
gmsh.write("cylinders.inp")


gmsh.fltk.run()
